In [1]:
from settings import *

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [3]:
ppfeatures = [
    "vesselId", 
    "time",
    # 'time_diff',
    'cog',
    'sog',
    'rot',
    'heading',
    # 'navstat',
    # 'etaRaw',
    'latitude',
    'longitude',
]

In [4]:
input_features = [
    "vesselId", 
    "time",
    'time_diff',
    'cog',
    'sog',
    'rot',
    'heading',
    # 'navstat',
    # 'etaRaw',
    'latitude',
    'longitude',
]

output_features = [
    "vesselId", 
    "time",
    # 'time_diff',
    'cog',
    'sog',
    'rot',
    'heading',
    # 'navstat',
    # 'etaRaw',
    'latitude',
    'longitude',
]

In [5]:
ais_train = pd.read_csv(AIS_TRAIN, sep='|')
ais_train['time'] = pd.to_datetime(ais_train['time'])

In [6]:
seq_len = 48

In [7]:
# vessels = pd.read_csv(VESSELS, sep='|')
# vessels.head()

In [8]:
# pd.read_csv(SCHEDULES_TO_MAY_2024, sep="|")

In [9]:
ais_test = pd.read_csv(AIS_TEST, sep=",")
ais_test['time'] = pd.to_datetime(ais_test['time'])

In [10]:
# ais_test.head()

In [11]:
len(ais_test["vesselId"].unique())

215

In [12]:

# def make_features(df: pd.DataFrame) -> pd.DataFrame:
#     raw = df.copy()

#     # time_diff: difference of time since last measure
#     raw['time'] = pd.to_datetime(raw['time'])
#     raw['time_diff'] = raw.sort_values(by=['vesselId', 'time']).groupby("vesselId")['time'].diff()
    
    
#     return raw

In [13]:
# ais_tr = make_features(ais_train)

In [14]:
# te.sort_values(by=["vesselId", "time"])

In [15]:
ais_train["portId"].isna().sum()

1615

In [16]:
ais_train

,time,cog,sog,rot,heading,navstat,etaRaw,latitude,longitude,vesselId,portId
0,2024-01-01 00:00:25,284.0,0.7,0,88,0,01-09 23:00,-34.74370,-57.85130,61e9f3a8b937134a3c4bfdf7,61d371c43aeaecc07011a37f
1,2024-01-01 00:00:36,109.6,0.0,-6,347,1,12-29 20:00,8.89440,-79.47939,61e9f3d4b937134a3c4bff1f,634c4de270937fc01c3a7689
2,2024-01-01 00:01:45,111.0,11.0,0,112,0,01-02 09:00,39.19065,-76.47567,61e9f436b937134a3c4c0131,61d3847bb7b7526e1adf3d19
3,2024-01-01 00:03:11,96.4,0.0,0,142,1,12-31 20:00,-34.41189,151.02067,61e9f3b4b937134a3c4bfe77,61d36f770a1807568ff9a126
4,2024-01-01 00:03:51,214.0,19.7,0,215,0,01-25 12:00,35.88379,-5.91636,61e9f41bb937134a3c4c0087,634c4de270937fc01c3a74f3
...,...,...,...,...,...,...,...,...,...,...,...
1522060,2024-05-07 23:59:07,359.1,13.4,0,1,0,05-08 05:00,52.19131,-5.82223,clh6aqawa0002gh0zypfa5dut,634c4de270937fc01c3a7417
1522061,2024-05-07 23:59:08,12.3,17.1,0,13,0,05-10 03:00,38.96142,-12.00502,61e9f3aeb937134a3c4bfe43,634c4de270937fc01c3a76a1
1522062,2024-05-07 23:59:08,269.8,14.9,-1,270,0,05-15 23:00,49.71372,-5.22042,61e9f43db937134a3c4c0169,634c4de270937fc01c3a787b
1522063,2024-05-07 23:59:08,8.0,18.7,0,6,0,05-08 12:45,38.27895,10.78280,61e9f469b937134a3c4c029b,61d3781293c6feb83e5eb73b


In [17]:
# CREATE time_diff AND MAKE IT IN SECONDS

train_vessel_id_time = ais_train[ppfeatures].copy()
train_vessel_id_time["split"] = "train"
train_vessel_id_time["ID"] = train_vessel_id_time.index

test_vessel_id_time = ais_test[["ID", "vesselId", "time" ]].copy()
test_vessel_id_time["split"] = "test"
ais_data = pd.concat([train_vessel_id_time, test_vessel_id_time], ignore_index=True)

# arrival time diff (from etaRaw)
# 
ais_data['time_diff'] = (
    ais_data
    .sort_values(by=['time'])
    .groupby("vesselId")['time']
    .diff()
    .dropna()
    .dt.total_seconds()
    .astype(int)
    .shift(-1)
)


def update_split_column(group: pd.Series) -> pd.Series:
    if group.name in ais_test["vesselId"].unique():
        group.iloc[-seq_len:] = "both"
    return group

df_temp = ais_data.copy()

ser_temp = pd.Series(
    df_temp[df_temp["split"]=="train"]
    .sort_values(by="time")
    .groupby("vesselId")["split"]
    .apply(update_split_column)
    .reset_index(drop=True)
)
ser_temp.index = df_temp[df_temp["split"]=="train"].sort_values(["vesselId", "time"]).index

ais_data.loc[ser_temp.index, "split"] = ser_temp

In [18]:
ais_data.describe()

,time,cog,sog,rot,heading,latitude,longitude,ID,time_diff
count,1573804,1.522065e+06,1.522065e+06,1.522065e+06,1.522065e+06,1.522065e+06,1.522065e+06,1.573804e+06,1.573115e+06
mean,2024-03-08 06:56:40.475442944,1.782494e+02,6.331703e+00,5.054561e-01,1.762737e+02,3.658497e+01,1.153646e+01,7.368634e+05,4.571780e+03
min,2024-01-01 00:00:25,0.000000e+00,0.000000e+00,-1.270000e+02,0.000000e+00,-4.753287e+01,-1.675409e+02,0.000000e+00,2.000000e+00
25%,2024-02-04 06:34:57,7.820000e+01,0.000000e+00,0.000000e+00,7.500000e+01,3.451683e+01,-5.212810e+00,3.417118e+05,1.146000e+03
50%,2024-03-09 12:33:56.500000,1.838000e+02,5.000000e-01,0.000000e+00,1.800000e+02,4.223370e+01,4.234350e+00,7.351625e+05,1.238000e+03
75%,2024-04-10 12:15:27.249999872,2.690000e+02,1.410000e+01,0.000000e+00,2.650000e+02,5.135197e+01,1.883715e+01,1.128613e+06,1.261000e+03
max,2024-05-12 23:59:58,3.600000e+02,1.023000e+02,1.280000e+02,5.110000e+02,7.055720e+01,1.788054e+02,1.522064e+06,5.923375e+06
std,NaN,1.076785e+02,7.539364e+00,1.760916e+01,1.070860e+02,2.290488e+01,6.871572e+01,4.515542e+05,5.228083e+04


In [19]:
ais_data

,vesselId,time,cog,sog,rot,heading,latitude,longitude,split,ID,time_diff
0,61e9f3a8b937134a3c4bfdf7,2024-01-01 00:00:25,284.0,0.7,0.0,88.0,-34.74370,-57.85130,train,0,NaN
1,61e9f3d4b937134a3c4bff1f,2024-01-01 00:00:36,109.6,0.0,-6.0,347.0,8.89440,-79.47939,train,1,NaN
2,61e9f436b937134a3c4c0131,2024-01-01 00:01:45,111.0,11.0,0.0,112.0,39.19065,-76.47567,train,2,NaN
3,61e9f3b4b937134a3c4bfe77,2024-01-01 00:03:11,96.4,0.0,0.0,142.0,-34.41189,151.02067,train,3,NaN
4,61e9f41bb937134a3c4c0087,2024-01-01 00:03:51,214.0,19.7,0.0,215.0,35.88379,-5.91636,train,4,NaN
...,...,...,...,...,...,...,...,...,...,...,...
1573799,61e9f3a8b937134a3c4bfdf3,2024-05-12 23:59:58,NaN,NaN,NaN,NaN,NaN,NaN,test,51734,1247.0
1573800,61e9f3b4b937134a3c4bfe77,2024-05-12 23:59:58,NaN,NaN,NaN,NaN,NaN,NaN,test,51735,1248.0
1573801,61e9f46cb937134a3c4c02b7,2024-05-12 23:59:58,NaN,NaN,NaN,NaN,NaN,NaN,test,51736,1243.0
1573802,61e9f465b937134a3c4c0269,2024-05-12 23:59:58,NaN,NaN,NaN,NaN,NaN,NaN,test,51737,1243.0


In [20]:
data_train = ais_data[(ais_data["split"]=="train")|(ais_data["split"]=="both")]
data_test = ais_data[(ais_data["split"]=="test")|(ais_data["split"]=="both")]

In [21]:
data_train

,vesselId,time,cog,sog,rot,heading,latitude,longitude,split,ID,time_diff
0,61e9f3a8b937134a3c4bfdf7,2024-01-01 00:00:25,284.0,0.7,0.0,88.0,-34.74370,-57.85130,train,0,NaN
1,61e9f3d4b937134a3c4bff1f,2024-01-01 00:00:36,109.6,0.0,-6.0,347.0,8.89440,-79.47939,train,1,NaN
2,61e9f436b937134a3c4c0131,2024-01-01 00:01:45,111.0,11.0,0.0,112.0,39.19065,-76.47567,train,2,NaN
3,61e9f3b4b937134a3c4bfe77,2024-01-01 00:03:11,96.4,0.0,0.0,142.0,-34.41189,151.02067,train,3,NaN
4,61e9f41bb937134a3c4c0087,2024-01-01 00:03:51,214.0,19.7,0.0,215.0,35.88379,-5.91636,train,4,NaN
...,...,...,...,...,...,...,...,...,...,...,...
1522060,clh6aqawa0002gh0zypfa5dut,2024-05-07 23:59:07,359.1,13.4,0.0,1.0,52.19131,-5.82223,both,1522060,1249.0
1522061,61e9f3aeb937134a3c4bfe43,2024-05-07 23:59:08,12.3,17.1,0.0,13.0,38.96142,-12.00502,both,1522061,1248.0
1522062,61e9f43db937134a3c4c0169,2024-05-07 23:59:08,269.8,14.9,-1.0,270.0,49.71372,-5.22042,train,1522062,900.0
1522063,61e9f469b937134a3c4c029b,2024-05-07 23:59:08,8.0,18.7,0.0,6.0,38.27895,10.78280,both,1522063,1260.0


In [22]:
scaled_features = [
    'time_diff',
    'cog',
    'sog',
    'rot',
    'heading',
    # 'navstat',
    # 'etaRaw',
    'latitude',
    'longitude',
]
scaled_features_y = [
    # 'time_diff',
    'cog',
    'sog',
    'rot',
    'heading',
    # 'navstat',
    # 'etaRaw',
    'latitude',
    'longitude',
]

In [23]:
scaler = StandardScaler()
scaler_y = StandardScaler()

In [24]:
data_train = data_train.dropna(subset="time_diff")

In [25]:
X = data_train.sort_values(["vesselId", "time"]).iloc[:-1][input_features]
y = data_train.sort_values(["vesselId", "time"]).iloc[1:][output_features]

In [26]:
# X[scaled_features] = scaler.fit_transform(X[scaled_features])
# y[scaled_features_y] = scaler_y.fit_transform(y[scaled_features_y])

In [27]:
# grouped = data_train.sort_values("time").groupby("vesselId")

# def create_sequences(data, sequence_length):
#     sequences = []
#     targets = []
#     for i in range(len(data) - sequence_length):
#         seq = data[i:i+sequence_length].values
#         target = data[scaled_features_y].iloc[i+sequence_length].values
#         sequences.append(seq)
#         targets.append(target)
#     return sequences, targets

# sequence_length = seq_len  # or any desired sequence length
# X, y = [], []

# for vessel_id, group in grouped:
#     sequences, targets = create_sequences(group[scaled_features], sequence_length)
#     X.extend(sequences)
#     y.extend(targets)

# # import numpy as np
# X = np.array(X)
# y = np.array(y)


In [28]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, shuffle=False)

In [29]:
X_train_scaled = X_train.copy()
X_train_scaled[scaled_features] = scaler.fit_transform(X_train_scaled[scaled_features])

y_train_scaled = y_train.copy()
y_train_scaled[scaled_features_y] = scaler_y.fit_transform(y_train_scaled[scaled_features_y])

In [30]:
X_test_scaled = X_test.copy()
X_test_scaled[scaled_features] = scaler.transform(X_test_scaled[scaled_features])

y_test_scaled = y_test.copy()
y_test_scaled[scaled_features_y] = scaler_y.transform(y_test_scaled[scaled_features_y])

In [31]:
X_train = X_train.sort_values(["time", "vesselId"])
y_train = y_train.sort_values(["time", "vesselId"])
X_test = X_test.sort_values(["time", "vesselId"])
y_test = y_test.sort_values(["time", "vesselId"])

In [32]:
import xgboost as xgb
from sklearn.multioutput import MultiOutputRegressor

from sklearn import preprocessing

from sklearn.model_selection import GridSearchCV

In [39]:
params = {
    # 'n_estimators': 5000,
    'gamma': 0.5,
    'subsample': 0.6,
    'n_estimators': 5000,
    'min_child_weight':  15,
    'colsample_bytree': 0.8,
    'max_depth': 4,
    'eta':  0.005,
    'refresh_leaf': 1,
}

In [40]:
param_grid = {
    'n_estimators': [5000, 10_000],
    'gamma': [0.5,],
    'subsample': [0.6,],
    'min_child_weight':  [15, 20],
    'colsample_bytree': [0.8,],
    'max_depth': [4, 10],
    'eta':  [0.005,],
    'refresh_leaf': [1,],

}

In [41]:
X_train = X_train_scaled[scaled_features]
y_train = y_train_scaled[scaled_features_y]
X_test = X_test_scaled[scaled_features]
y_test = y_test_scaled[scaled_features_y]

In [42]:
model = xgb.XGBRegressor(**params)
xgb_model = MultiOutputRegressor(model)

# grid_search = GridSearchCV(xgb_model, param_grid=param_grid)
# grid_search.fit(
#     X_train, y_train,
#     eval_set=[(X_train, y_train), (X_test, y_test)]
# )
# xgb_model = grid_search.best_estimator_
# print(grid_search.best_params_)

In [37]:
X_train

,time_diff,cog,sog,rot,heading,latitude,longitude
131279,-0.070750,1.213621,1.491198,0.270202,1.275712,-1.182030,0.958324
131514,-0.062987,1.206163,1.437532,0.270202,1.266358,-1.178781,0.956895
131696,-0.064069,1.216417,1.437532,0.328946,1.275712,-1.176166,0.955762
131885,-0.055300,1.208027,1.357033,0.387689,1.275712,-1.173684,0.954658
132038,-0.070750,1.213621,1.330200,0.270202,1.275712,-1.171966,0.953889
...,...,...,...,...,...,...,...
543944,-0.074148,-0.442030,-0.829850,-0.023516,1.219587,0.208572,0.279065
544247,-0.083486,-0.442030,-0.829850,-0.023516,1.228941,0.208572,0.279065
544906,-0.077583,-0.442030,-0.829850,-0.023516,1.228941,0.208572,0.279065
545087,-0.000675,-0.442030,-0.829850,-0.023516,1.228941,0.208572,0.279065


In [43]:
xgb_model.fit(
    X_train, y_train,
    eval_set=[(X_train, y_train), (X_test, y_test)],
    # early_stopping_rounds=50,
    verbose=False,
)
# xgb_model.load_model("models/xgb_model_v2.json")

XGBoostError: [00:00:44] /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:50: Check failed: info.labels.Size() == preds.Size() (1217100 vs. 7302600) : Invalid shape of labels.
Stack trace:
  [bt] (0) 1   libxgboost.dylib                    0x00000003936ac58c dmlc::LogMessageFatal::~LogMessageFatal() + 124
  [bt] (1) 2   libxgboost.dylib                    0x0000000393930930 xgboost::obj::(anonymous namespace)::CheckRegInputs(xgboost::MetaInfo const&, xgboost::HostDeviceVector<float> const&) + 224
  [bt] (2) 3   libxgboost.dylib                    0x0000000393930014 xgboost::obj::RegLossObj<xgboost::obj::LinearSquareLoss>::GetGradient(xgboost::HostDeviceVector<float> const&, xgboost::MetaInfo const&, int, xgboost::linalg::Tensor<xgboost::detail::GradientPairInternal<float>, 2>*) + 64
  [bt] (3) 4   libxgboost.dylib                    0x0000000393897a44 xgboost::LearnerImpl::UpdateOneIter(int, std::__1::shared_ptr<xgboost::DMatrix>) + 652
  [bt] (4) 5   libxgboost.dylib

In [40]:
try:
    xgb_model.save_model("models/xgb_model_v3_1.json")
except:
    xgb_model.save_model("models/xgb_model_v3_2.json")

In [41]:
# import matplotlib.pyplot as plt

# results = xgb_model.evals_result()
# train_errors = results['validation_0']['rmse']
# test_errors = results['validation_1']['rmse']

# plt.figure(figsize=(10, 6))
# plt.plot(train_errors, label='Train')
# plt.plot(test_errors, label='Test')
# plt.xlabel('Boosting Rounds')
# plt.ylabel("rmse")
# plt.legend()
# plt.title('Learning Curves')
# plt.show()

In [42]:
y_pred = xgb_model.predict(X_test)

In [ ]:
len_test = len(y_test)
len_test

In [44]:
df_y = y.iloc[-len_test:]

In [ ]:
y_test

In [ ]:
y_test.columns

## Visualize

In [ ]:
df_y = y.iloc[-len_test:].copy()
df_y = df_y.reset_index()
vessel_id = df_y["vesselId"].unique()[104]
print("vesselId:", vessel_id)
index = list(df_y[df_y["vesselId"]==vessel_id].index)

y_test_plot = y_test.reset_index().iloc[index]
y_pred_plot = pd.DataFrame(y_pred.take(index, axis=0), columns=y_test.columns)

plt.figure(figsize=(20, 10))
plt.subplot(1, 2, 1)
plt.plot(y_test_plot["longitude"], y_test_plot["latitude"], 'o', label="test")
plt.plot(y_pred_plot["longitude"], y_pred_plot["latitude"], 'o', label="pred")
plt.legend()
plt.subplot(1, 2, 2)
plt.plot(y_test_plot["longitude"], y_test_plot["latitude"], label="test")
plt.plot(y_pred_plot["longitude"], y_pred_plot["latitude"], label="pred")
plt.legend()

In [48]:

# vessel_id = y_test.values

In [49]:
# from src.eval.metric import *

# score(y_test[["ID", "latitude", "longitude"]], y_pred, "ID")

## Forecast

In [50]:

def iterative_forecast(last_known, model, steps, sequence_length):
    predicted = []
    current_sequence = last_known[:sequence_length] #.reshape(1,-1)
    # current_sequence = last_known[-sequence_length:]
    for k in range(steps):
        # next_pred = model.predict(current_sequence.reshape(1, sequence_length, -1))[0]
        next_pred = model.predict(current_sequence)[0]
        # TODO: use ais_test["time_diff"] (not predicted currently)
        predicted.append(next_pred)
        last_known[k+1,1:] = next_pred
        # Update current_sequence by appending next prediction
        current_sequence = last_known[k+1].reshape(1,-1)
    
    return predicted

In [51]:
grouped_test = data_test.groupby("vesselId")

In [ ]:
predictions = []
forecast_steps = 300
sequence_length = seq_len

for vessel_id, group in grouped_test:
    forecast_steps = len(group['time'].values) - 1

    # last_known_features = scaler.transform(group[scaled_features].values[-sequence_length:])
    last_known_features = scaler.transform(group[scaled_features].values)
    future_preds = iterative_forecast(last_known_features, xgb_model, forecast_steps, sequence_length)
    
    # Store the predictions
    df_pred = pd.DataFrame(scaler_y.inverse_transform(future_preds), columns=scaled_features_y)
    df_pred['time'] = group['time'].iloc[1:].values
    df_pred["vesselId"] = vessel_id
    predictions.append(df_pred)


In [ ]:
df_pred.describe()

In [ ]:
group[scaled_features].values[-sequence_length:]

In [ ]:
last_known_features

In [56]:
df_preds = pd.concat(predictions, ignore_index=True)

In [ ]:
df_preds.describe()

In [58]:
res = pd.merge(ais_test, df_preds[["vesselId","time", "latitude", "longitude"]],on=["time", "vesselId"], how="left")

In [59]:
res["longitude_predicted"] = res["longitude"]
res["latitude_predicted"] = res["latitude"]
# res["id"] = res["ID"]
res = res.drop(columns=["longitude", "latitude"])

In [ ]:
res[["ID","longitude_predicted","latitude_predicted"]]

In [ ]:
from utils import *

from settings import *

from datetime import datetime
import uuid

import pandas as pd

def make_file_name() -> str:
    file_name = str(uuid.uuid4()) + ".csv"
    print(f"Submission file name is: {file_name}")
    return file_name

def submit(forecast: pd.DataFrame, file_name: str = None) -> None:
    sample_submission = pd.read_csv(AIS_SAMPLE_SUBMISSION)
    file_name = file_name if file_name else make_file_name()

    repertory = SUBMISSION_FODLER.joinpath(file_name)
    sample_submission = sample_submission[['ID']].merge(forecast[["ID","longitude_predicted","latitude_predicted"]], on='ID', how='left')
    try:
        sample_submission.to_csv(repertory, index=False)
    except:
        print("Error register file")
        submit(forecast)

submit(res)

In [ ]:
res["longitude_predicted"].plot()

In [ ]:
y_test["longitude"].plot()